In [ ]:
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from copy import deepcopy
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import torchvision
import torchvision.transforms as transforms

# 数据预处理：resize 和 normalization
transform = transforms.Compose([
    transforms.Resize(224),  # 因为 VGG 输入是 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载训练集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# 加载测试集
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:13<00:00, 12.5MB/s]


In [ ]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 2. 挂载 Google Drive（如果还没挂载的话）
from google.colab import drive
drive.mount('/content/drive')
print(device)

Mounted at /content/drive
cuda


In [ ]:
basemodel = models.vgg16()  # 或者你自定义的 VGG19-CIFAR 版本

# 如果你之前做了修改（比如分类数不同），需要一致设置：
basemodel.classifier[6] = torch.nn.Linear(4096, 10)  # CIFAR-10 分类任务

# 4. 加载参数
save_path = "/content/drive/MyDrive/模型/VGG16_CIFAR.pth"
basemodel.load_state_dict(torch.load(save_path))

<All keys matched successfully>

In [ ]:
print(basemodel)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
# 测试函数
def test_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            pred = output.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

In [ ]:
# 剪枝率列表
prune_rates = [0.1, 0.3, 0.5, 0.7, 0.9]

# 找到所有Conv层
conv_layers = [(name, m) for name, m in basemodel.features.named_modules() if isinstance(m, nn.Conv2d)]
print("-----VGG16-----")

# 对每个Conv层进行多种剪枝测试
for i, (layer_name, layer) in enumerate(conv_layers):
    for rate in prune_rates:
        # 深拷贝模型
        model = deepcopy(basemodel)
        model.to(device)
        model.eval()
        # 获取当前层对象
        target_layer = dict(model.features.named_modules())[layer_name]
        # 执行剪枝
        prune.l1_unstructured(target_layer, name='weight', amount=rate)
        # 评估剪枝后模型的准确率
        acc = test_accuracy(model, testloader)
        print(f"[Layer {i}] Pruned {layer_name} with {int(rate*100)}% -> Accuracy: {acc:.2f}%")

-----VGG16-----
[Layer 0] Pruned 0 with 10% -> Accuracy: 86.70%
[Layer 0] Pruned 0 with 30% -> Accuracy: 85.67%
[Layer 0] Pruned 0 with 50% -> Accuracy: 81.76%
[Layer 0] Pruned 0 with 70% -> Accuracy: 63.25%
[Layer 0] Pruned 0 with 90% -> Accuracy: 20.80%
[Layer 1] Pruned 2 with 10% -> Accuracy: 86.54%
[Layer 1] Pruned 2 with 30% -> Accuracy: 86.48%
[Layer 1] Pruned 2 with 50% -> Accuracy: 86.46%
[Layer 1] Pruned 2 with 70% -> Accuracy: 84.84%
[Layer 1] Pruned 2 with 90% -> Accuracy: 66.55%
[Layer 2] Pruned 5 with 10% -> Accuracy: 86.58%
[Layer 2] Pruned 5 with 30% -> Accuracy: 86.58%
[Layer 2] Pruned 5 with 50% -> Accuracy: 86.39%
[Layer 2] Pruned 5 with 70% -> Accuracy: 85.58%
[Layer 2] Pruned 5 with 90% -> Accuracy: 76.48%
[Layer 3] Pruned 7 with 10% -> Accuracy: 86.57%
[Layer 3] Pruned 7 with 30% -> Accuracy: 86.56%
[Layer 3] Pruned 7 with 50% -> Accuracy: 85.57%
[Layer 3] Pruned 7 with 70% -> Accuracy: 83.82%
[Layer 3] Pruned 7 with 90% -> Accuracy: 62.62%
[Layer 4] Pruned 10 with

In [ ]:
model = models.vgg11()
model.classifier[6] = nn.Linear(4096,10)

save_path = "/content/drive/MyDrive/模型/VGG11_CIFAR.pth"
model.load_state_dict(torch.load(save_path))


<All keys matched successfully>

In [ ]:
model = model.to(device)

In [ ]:
acc = test_accuracy(model,testloader)
acc

86.06

In [ ]:
def count_conv_params(model):
    total = 0
    for module in model.features.modules():
        if isinstance(module, torch.nn.Conv2d):
            total += sum(p.numel() for p in module.parameters())
    return total

print(f"Total Conv2d parameters (before pruning): {count_conv_params(model)}")

Total Conv2d parameters (before pruning): 9220480


In [ ]:
# 收集所有 Conv2d 层
parameters_to_prune = []
for name, module in model.features.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        parameters_to_prune.append((module, 'weight'))

# 执行全局剪枝（50%）
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.6  # 剪枝 50%---->84%---->稀疏度3.58%
)
#128807306卷积：9220480
global_acc = test_accuracy(model,testloader)

print(f"剪枝之后，准确率为:{global_acc}，未剪枝的时候参数数量为{count_conv_params(model)}")

剪枝之后，准确率为:79.97，未剪枝的时候参数数量为9220480


In [ ]:
def count_zero_weights_pruned(model):
    total, zeros = 0, 0
    for name, module in model.named_modules():
        if hasattr(module, "weight"):
            w = module.weight.detach()  # 这是带 mask 的权重
            total += w.numel()
            zeros += torch.sum(w == 0).item()
    return total, zeros, 100 * zeros / total

total, zeros, zero_ratio = count_zero_weights_pruned(model)
print(f"Total weights: {total}")
print(f"Zero weights: {zeros}")
print(f"Sparsity: {zero_ratio:.2f}%")


Total weights: 128796352
Zero weights: 5530637
Sparsity: 4.29%


In [ ]:
4608864/128796352

0.03578411910300068